<a href="https://colab.research.google.com/github/rickvelloso/Learning-IA/blob/main/case_LocPay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
from collections import Counter
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, recall_score


In [16]:
df = pd.read_csv('Loan_default.csv')
df = df.dropna(subset=['Default'])
y = df['Default']
X = df.drop(['LoanID', 'Default'], axis=1)
X


,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner
0,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes
1,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes
2,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No
3,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No
4,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,19,37979,210682,541,109,4,14.11,12,0.85,Bachelor's,Full-time,Married,No,No,Other,No
255343,32,51953,189899,511,14,2,11.55,24,0.21,High School,Part-time,Divorced,No,No,Home,No
255344,56,84820,208294,597,70,3,5.29,60,0.50,High School,Self-employed,Married,Yes,Yes,Auto,Yes
255345,42,85109,60575,809,40,1,20.90,48,0.44,High School,Part-time,Single,Yes,Yes,Other,No


In [12]:
X_processed = pd.get_dummies(X, drop_first=True)

In [13]:
print(f"Formato original de X: {X.shape}")
print(f"Formato de X após One-Hot Encoding: {X_processed.shape}")
print("-" * 40)

Formato original de X: (255347, 16)
Formato de X após One-Hot Encoding: (255347, 24)
----------------------------------------


In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X_processed,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print("Dados prontos para o modelo!")
print(f"Formato do X de Treino (escalado): {X_train_scaled.shape}")
print(f"Formato do X de Teste (escalado): {X_test_scaled.shape}")

Dados prontos para o modelo!
Formato do X de Treino (escalado): (178742, 24)
Formato do X de Teste (escalado): (76605, 24)


In [ ]:
X_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Age                           255347 non-null  int64  
 1   Income                        255347 non-null  int64  
 2   LoanAmount                    255347 non-null  int64  
 3   CreditScore                   255347 non-null  int64  
 4   MonthsEmployed                255347 non-null  int64  
 5   NumCreditLines                255347 non-null  int64  
 6   InterestRate                  255347 non-null  float64
 7   LoanTerm                      255347 non-null  int64  
 8   DTIRatio                      255347 non-null  float64
 9   Education_High School         255347 non-null  bool   
 10  Education_Master's            255347 non-null  bool   
 11  Education_PhD                 255347 non-null  bool   
 12  EmploymentType_Part-time      255347 non-nul

In [ ]:
print(X_processed.iloc[:,].head())

   Age  Income  LoanAmount  CreditScore  MonthsEmployed  NumCreditLines  \
0   56   85994       50587          520              80               4   
1   69   50432      124440          458              15               1   
2   46   84208      129188          451              26               3   
3   32   31713       44799          743               0               3   
4   60   20437        9139          633               8               4   

   InterestRate  LoanTerm  DTIRatio  Education_High School  ...  \
0         15.23        36      0.44                  False  ...   
1          4.81        60      0.68                  False  ...   
2         21.17        24      0.31                  False  ...   
3          7.07        24      0.23                   True  ...   
4          6.51        48      0.73                  False  ...   

   EmploymentType_Unemployed  MaritalStatus_Married  MaritalStatus_Single  \
0                      False                  False                 F

In [ ]:
model = LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [ ]:
y_pred_padrao = model.predict(X_test_scaled)

In [ ]:
y_probabilidades = model.predict_proba(X_test_scaled)[:, 1]

In [ ]:
print("--- MODELO 1: PRAGMÁTICO (Limiar Padrão 0.5) ---")

cm_padrao = confusion_matrix(y_test, y_pred_padrao)
print("Matriz de Confusão (Padrão):")
print(cm_padrao)
# Lembrete da Matriz:
# [[TN (Bom/Bom)    FP (Bom/Ruim - Atrito)]
#  [FN (Ruim/Bom - Perda) TP (Ruim/Ruim)]]

print("\nRelatório de Classificação (Padrão):")
print(classification_report(y_test, y_pred_padrao))

--- MODELO 1: PRAGMÁTICO (Limiar Padrão 0.5) ---
Matriz de Confusão (Padrão):
[[45720 21989]
 [ 2684  6212]]

Relatório de Classificação (Padrão):
              precision    recall  f1-score   support

           0       0.94      0.68      0.79     67709
           1       0.22      0.70      0.33      8896

    accuracy                           0.68     76605
   macro avg       0.58      0.69      0.56     76605
weighted avg       0.86      0.68      0.73     76605



In [ ]:
print("\n--- MODELO 2: CONSERVADOR (Limiar Baixo 0.2) ---")

LIMIAR_CONSERVADOR = 0.2 # Chute inicial

# Se a probabilidade de ser 'Ruim' (1) for > 20%, já preveja 'Ruim'
y_pred_conservador = np.where(y_probabilidades > LIMIAR_CONSERVADOR, 1, 0)

cm_conservador = confusion_matrix(y_test, y_pred_conservador)
print("Matriz de Confusão (Conservador):")
print(cm_conservador)

print("\nRelatório de Classificação (Conservador):")
print(classification_report(y_test, y_pred_conservador))


--- MODELO 2: CONSERVADOR (Limiar Baixo 0.2) ---
Matriz de Confusão (Conservador):
[[11039 56670]
 [  236  8660]]

Relatório de Classificação (Conservador):
              precision    recall  f1-score   support

           0       0.98      0.16      0.28     67709
           1       0.13      0.97      0.23      8896

    accuracy                           0.26     76605
   macro avg       0.56      0.57      0.26     76605
weighted avg       0.88      0.26      0.27     76605



In [ ]:
print("\n--- MODELO 3: AGRESSIVO (Limiar Alto 0.8) ---")

LIMIAR_AGRESSIVO = 0.8 # Chute inicial

# Só preveja 'Ruim' (1) se tiver mais de 80% de certeza
y_pred_agressivo = np.where(y_probabilidades > LIMIAR_AGRESSIVO, 1, 0)

cm_agressivo = confusion_matrix(y_test, y_pred_agressivo)
print("Matriz de Confusão (Agressivo):")
print(cm_agressivo)

print("\nRelatório de Classificação (Agressivo):")
print(classification_report(y_test, y_pred_agressivo))


--- MODELO 3: AGRESSIVO (Limiar Alto 0.8) ---
Matriz de Confusão (Agressivo):
[[66084  1625]
 [ 7526  1370]]

Relatório de Classificação (Agressivo):
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     67709
           1       0.46      0.15      0.23      8896

    accuracy                           0.88     76605
   macro avg       0.68      0.57      0.58     76605
weighted avg       0.85      0.88      0.85     76605



Dado o teste com o modelo de regressão linear agora vou rodar um 'autoML' (com as limitações do ambiente de execução ele será mais simples do que para um ambiente profissional!) e comparar com outros diversos modelos.

In [ ]:
counter = Counter(y_train)
scale_pos_weight = counter[0] / counter[1]
print(f"Usando scale_pos_weight para XGBoost: {scale_pos_weight:.2f}")

Usando scale_pos_weight para XGBoost: 7.61


In [ ]:
models = {
    "LogisticRegression": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
    "RandomForest": RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight, n_jobs=-1)
}

In [ ]:
results = {}
for name, model in models.items():
    print(f"\n--- Treinando e Avaliando: {name} ---")
    start_time = time.time()

    # Treinar
    model.fit(X_train_scaled, y_train)

    # Prever
    y_pred = model.predict(X_test_scaled)

    # Calcular métricas
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    results[name] = (cm, report, time.time() - start_time)

    print(f"Tempo de Treino/Previsão: {results[name][2]:.2f}s")
    print("Matriz de Confusão:")
    print(cm)
    print("\nRelatório de Classificação:")
    print(report)
    print("-" * 50)

print("--- Competição Finalizada ---")


--- Treinando e Avaliando: LogisticRegression ---
Tempo de Treino/Previsão: 0.37s
Matriz de Confusão:
[[45720 21989]
 [ 2684  6212]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.94      0.68      0.79     67709
           1       0.22      0.70      0.33      8896

    accuracy                           0.68     76605
   macro avg       0.58      0.69      0.56     76605
weighted avg       0.86      0.68      0.73     76605

--------------------------------------------------

--- Treinando e Avaliando: RandomForest ---
Tempo de Treino/Previsão: 40.90s
Matriz de Confusão:
[[67649    60]
 [ 8763   133]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     67709
           1       0.69      0.01      0.03      8896

    accuracy                           0.88     76605
   macro avg       0.79      0.51      0.48     76605
weighted avg       0.86      

Com esses resultados fica claro que ainda não obtivemos nenhum resultado muito proveitoso, visto as métricas de recall.
Foi possivel também verificar que o problema não estava no modelo em si, então pode haver um problema com o desbalanceamento dos dados (que é comum em dados para verificação de fraudes, uma vez que representam a minoria dos casos), para isso eu tentei utilizar o proprio modelo tratar os dados desbalanceados (com o class_weight='balanced' (serve para o proprio modelo entender que os dados não estão balanceados e então tentar por si balancear)).
Agora vou utilizar o SMOTENC para balancear os dados (algoritmo que usa técnica de over-sampling para criar os dados sintéticos das classes menores para a base ficar equilibrada 50%).

In [ ]:
categorical_features_indices = [
    i for i, dtype in enumerate(X_processed.dtypes)
    if dtype == 'uint8' or dtype == 'bool'
]

In [ ]:
sm = SMOTENC(random_state=42, categorical_features=categorical_features_indices)
print("Rodando o SMOTENC... (deve demorar um pouco)")
start_smote = time.time()
X_train_smote, y_train_smote = sm.fit_resample(X_train_scaled, y_train)
end_smote = time.time()
print(f"SMOTENC concluído em {end_smote - start_smote:.2f}s")

Rodando o SMOTENC... (deve demorar um pouco)
SMOTENC concluído em 85.31s


In [ ]:
print(f"Formato original de y_train: {Counter(y_train)}")
print(f"Formato NOVO de y_train (SMOTE): {Counter(y_train_smote)}")
print("--- Balanceamento Concluído ---")

models_smote = {
    "LogisticRegression (SMOTE)": LogisticRegression(random_state=42, max_iter=1000),
    "RandomForest (SMOTE)": RandomForestClassifier(random_state=42, n_jobs=-1),
    "XGBoost (SMOTE)": xgb.XGBClassifier(random_state=42, n_jobs=-1)
}

Formato original de y_train: Counter({0: 157985, 1: 20757})
Formato NOVO de y_train (SMOTE): Counter({0: 157985, 1: 157985})
--- Balanceamento Concluído ---


In [ ]:
results_smote = {}
for name, model in models_smote.items():
    print(f"\n--- Treinando e Avaliando (SMOTE): {name} ---")
    start_time = time.time()

    model.fit(X_train_smote, y_train_smote)

    y_pred_smote = model.predict(X_test_scaled)

    cm_smote = confusion_matrix(y_test, y_pred_smote)
    report_smote = classification_report(y_test, y_pred_smote)

    results_smote[name] = (cm_smote, report_smote, time.time() - start_time)

    print(f"Tempo de Treino/Previsão: {results_smote[name][2]:.2f}s")
    print("Matriz de Confusão:")
    print(cm_smote)
    print("\nRelatório de Classificação:")
    print(report_smote)
    print("-" * 50)

print("--- Competição (SMOTE) Finalizada ---")


--- Treinando e Avaliando (SMOTE): LogisticRegression (SMOTE) ---
Tempo de Treino/Previsão: 0.65s
Matriz de Confusão:
[[47877 19832]
 [ 3197  5699]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.94      0.71      0.81     67709
           1       0.22      0.64      0.33      8896

    accuracy                           0.70     76605
   macro avg       0.58      0.67      0.57     76605
weighted avg       0.85      0.70      0.75     76605

--------------------------------------------------

--- Treinando e Avaliando (SMOTE): RandomForest (SMOTE) ---
Tempo de Treino/Previsão: 106.30s
Matriz de Confusão:
[[62325  5384]
 [ 6315  2581]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.91      0.92      0.91     67709
           1       0.32      0.29      0.31      8896

    accuracy                           0.85     76605
   macro avg       0.62      0.61      0.61     76

In [ ]:
from sklearn.model_selection import GridSearchCV
print("--- Iniciando a Busca Otimizada (SMOTE + Recall) ---")

recall_c1_scorer = make_scorer(recall_score, pos_label=1)


params_lr = {
    'C': [0.1, 1.0, 10.0] # Testa diferentes forças de regularização
}

params_rf = {
    'n_estimators': [100],
    'max_depth': [5, 10, None],
    'class_weight': ['balanced', None] # Testar se 'balanced' ajuda MESMO com SMOTE
}


grid_lr = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000),
    params_lr,
    scoring=recall_c1_scorer,
    cv=3, # 3-fold CV (mais rápido)
    n_jobs=-1
)

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    params_rf,
    scoring=recall_c1_scorer,
    cv=3,
    n_jobs=-1
)

print("\nBuscando pelo melhor LogisticRegression (SMOTE + Recall)...")
start_time = time.time()
grid_lr.fit(X_train_smote, y_train_smote)
print(f"LR treinado em {time.time() - start_time:.2f}s")

print("\nBuscando pelo melhor RandomForest (SMOTE + Recall)...")
start_time = time.time()
grid_rf.fit(X_train_smote, y_train_smote)
print(f"RF treinado em {time.time() - start_time:.2f}s")

print("\n--- Resultados Finais no Teste ---")

y_pred_lr_auto = grid_lr.best_estimator_.predict(X_test_scaled)
print("\nMelhor LogisticRegression (Otimizado para Recall C1):")
print(f"Melhores Parâmetros: {grid_lr.best_params_}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_lr_auto))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_lr_auto))

y_pred_rf_auto = grid_rf.best_estimator_.predict(X_test_scaled)
print("\nMelhor RandomForest (Otimizado para Recall C1):")
print(f"Melhores Parâmetros: {grid_rf.best_params_}")
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_rf_auto))
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred_rf_auto))

--- Iniciando a Busca Otimizada (SMOTE + Recall) ---

Buscando pelo melhor LogisticRegression (SMOTE + Recall)...
LR treinado em 12.50s

Buscando pelo melhor RandomForest (SMOTE + Recall)...
RF treinado em 881.53s

--- Resultados Finais no Teste ---

Melhor LogisticRegression (Otimizado para Recall C1):
Melhores Parâmetros: {'C': 0.1}
Matriz de Confusão:
[[47879 19830]
 [ 3196  5700]]
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.94      0.71      0.81     67709
           1       0.22      0.64      0.33      8896

    accuracy                           0.70     76605
   macro avg       0.58      0.67      0.57     76605
weighted avg       0.85      0.70      0.75     76605


Melhor RandomForest (Otimizado para Recall C1):
Melhores Parâmetros: {'class_weight': None, 'max_depth': None, 'n_estimators': 100}
Matriz de Confusão:
[[62325  5384]
 [ 6315  2581]]
Relatório de Classificação:
              precision    recall  f1-score  

Após uma investigação exaustiva, esta análise de protótipo chega a uma conclusão fundamental:

Mesmo após testar múltiplos modelos (LogisticRegression, RandomForest, XGBoost), aplicar duas estratégias distintas de balanceamento de dados (class_weight e SMOTENC) e realizar o tuning de hiperparâmetros (GridSearchCV), não foi o suficiente para atingir um nível de recall de inadimplência (Classe 1) aceitável para o negócio.

O melhor resultado obtido, um Recall de 64% (do modelo LogisticRegression + SMOTE), ainda é um indicador de alto risco para a operação.

E esse pode ser um problema que vocês, na LocPay, estão passando (ou passarão) na análise de crédito ao tentar escalar.

Por que, neste exemplo, não foi possível atingir 85% ou 90% de recall?

A resposta não está no modelo, mas nos dados. Os dados coletados são preditores fracos. Apesar de termos 18 colunas (que se expandiram para 24+ após o One-Hot Encoding), os atributos (Idade, Renda, Nível Educacional) não são suficientes para prever o comportamento financeiro de um cliente.

Para uma previsão de risco robusta, o caminho não é otimizar mais os modelos. O caminho é o Enriquecimento de Dados: usar o "Módulo de Scoring" proposto para consultar APIs externas de bureaus de crédito (ex: Serasa, SPC) e Open Banking. Somente assim poderemos capturar o padrão real do cliente e atingir o nível de segurança que o negócio exige.